# Plagiarism Detection Model

Now that you've created training and test data, you are ready to define and train a model. Your goal in this notebook, will be to train a binary classification model that learns to label an answer file as either plagiarized or not, based on the features you provide the model.

This task will be broken down into a few discrete steps:

* Upload your data to S3.
* Define a binary classification model and a training script.
* Train your model and deploy it.
* Evaluate your deployed classifier and answer some questions about your approach.

To complete this notebook, you'll have to complete all given exercises and answer all the questions in this notebook.
> All your tasks will be clearly labeled **EXERCISE** and questions as **QUESTION**.

It will be up to you to explore different classification models and decide on a model that gives you the best performance for this dataset.

---

## Load Data to S3

In the last notebook, you should have created two files: a `training.csv` and `test.csv` file with the features and class labels for the given corpus of plagiarized/non-plagiarized text data. 

>The below cells load in some AWS SageMaker libraries and creates a default bucket. After creating this bucket, you can upload your locally stored data to S3.

Save your train and test `.csv` feature files, locally. To do this you can run the second notebook "2_Plagiarism_Feature_Engineering" in SageMaker or you can manually upload your files to this notebook using the upload icon in Jupyter Lab. Then you can upload local files to S3 by using `sagemaker_session.upload_data` and pointing directly to where the training data is saved.

In [1]:
E I had endpoint errors. This install of the previous version of sagemaker==1.72.0 solved
!pip install sagemaker==1.72.0  # need to kill and restart kernel

In [19]:
import pandas as pd
import boto3
import sagemaker

In [20]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# create an S3 bucket
bucket = sagemaker_session.default_bucket()

## EXERCISE: Upload your training data to S3

Specify the `data_dir` where you've saved your `train.csv` file. Decide on a descriptive `prefix` that defines where your data will be uploaded in the default S3 bucket. Finally, create a pointer to your training data by calling `sagemaker_session.upload_data` and passing in the required parameters. It may help to look at the [Session documentation](https://sagemaker.readthedocs.io/en/stable/session.html#sagemaker.session.Session.upload_data) or previous SageMaker code examples.

You are expected to upload your entire directory. Later, the training script will only access the `train.csv` file.

In [21]:
# should be the name of directory you created to save your features data
import os # Had to import os because sagemaker states it was not defined

data_dir = 'plagiarism_data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    
session = sagemaker.Session() # Store the current SageMaker session

# set prefix, a descriptive name for a directory  
prefix = 'Training_a_plagiarism_model'
# upload all data to S3
train_location = session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)

### Test cell

Test that your data has been successfully uploaded. The below cell prints out the items in your S3 bucket and will throw an error if it is empty. You should see the contents of your `data_dir` and perhaps some checkpoints. If you see any other files listed, then you may have some old model files that you can delete via the S3 console (though, additional files shouldn't affect the performance of model developed in this notebook).

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# confirm that data is in S3 bucket
empty_check = []
for obj in boto3.resource('s3').Bucket(bucket).objects.all():
    empty_check.append(obj.key)
    print(obj.key)

assert len(empty_check) !=0, 'S3 bucket is empty.'
print('Test passed!')

Training_a_plagiarism_model/test.csv
Training_a_plagiarism_model/train.csv
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/profiler-output/system/incremental/2021051202/1620787800.algo-1.json
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/profiler-output/system/incremental/2021051202/1620787860.algo-1.json
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/rule-output/ProfilerReport-1620787622/profiler-output/profiler-report.html
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/rule-output/ProfilerReport-1620787622/profiler-output/profiler-report.ipynb
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/rule-output/ProfilerReport-1620787622/profiler-output/profiler-reports/BatchSize.json
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/rule-output/ProfilerReport-1620787622/profiler-output/profiler-reports/CPUBottleneck.json
plagiarism_data/pytorch-training-2021-05-12-02-47-02-075/rule-output/ProfilerReport-1620787622/profiler-output/profile

---

# Modeling

Now that you've uploaded your training data, it's time to define and train a model!

The type of model you create is up to you. For a binary classification task, you can choose to go one of three routes:
* Use a built-in classification algorithm, like LinearLearner.
* Define a custom Scikit-learn classifier, a comparison of models can be found [here](https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html).
* Define a custom PyTorch neural network classifier. 

It will be up to you to test out a variety of models and choose the best one. Your project will be graded on the accuracy of your final model. 
 
---

## EXERCISE: Complete a training script 

To implement a custom classifier, you'll need to complete a `train.py` script. You've been given the folders `source_sklearn` and `source_pytorch` which hold starting code for a custom Scikit-learn model and a PyTorch model, respectively. Each directory has a `train.py` training script. To complete this project **you only need to complete one of these scripts**; the script that is responsible for training your final model.

A typical training script:
* Loads training data from a specified directory
* Parses any training & model hyperparameters (ex. nodes in a neural network, training epochs, etc.)
* Instantiates a model of your design, with any specified hyperparams
* Trains that model 
* Finally, saves the model so that it can be hosted/deployed, later

### Defining and training a model
Much of the training script code is provided for you. Almost all of your work will be done in the `if __name__ == '__main__':` section. To complete a `train.py` file, you will:
1. Import any extra libraries you need
2. Define any additional model training hyperparameters using `parser.add_argument`
2. Define a model in the `if __name__ == '__main__':` section
3. Train the model in that same section

Below, you can use `!pygmentize` to display an existing `train.py` file. Read through the code; all of your tasks are marked with `TODO` comments. 

**Note: If you choose to create a custom PyTorch model, you will be responsible for defining the model in the `model.py` file,** and a `predict.py` file is provided. If you choose to use Scikit-learn, you only need a `train.py` file; you may import a classifier from the `sklearn` library.

In [23]:
# directory can be changed to: source_sklearn or source_pytorch
!pygmentize source_sklearn/train.py

from __future__ import print_function

import argparse
import os
import pandas as pd

# sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. 
# from sklearn.externals import joblib
# Import joblib package directly
import joblib

## TODO: Import any additional libraries you need to define a model


# Provided model load function
def model_fn(model_dir):
    """Load model from the model_dir. This is the same model that is saved
    in the main if statement.
    """
    print("Loading model.")
    
    # load using joblib
    model = joblib.load(os.path.join(model_dir, "model.joblib"))
    print("Done loading model.")
    
    return model


## TODO: Complete the main code
if __name__ == '__main__':
    
    # All of the model parameters and training parameters are sent as arguments
    # when this script is executed, during a training job
    
    # Here we set up an argument parser to easily access the parameters
    parser = argparse.ArgumentParser()

    # SageM

### Provided code

If you read the code above, you can see that the starter code includes a few things:
* Model loading (`model_fn`) and saving code
* Getting SageMaker's default hyperparameters
* Loading the training data by name, `train.csv` and extracting the features and labels, `train_x`, and `train_y`

If you'd like to read more about model saving with [joblib for sklearn](https://scikit-learn.org/stable/modules/model_persistence.html) or with [torch.save](https://pytorch.org/tutorials/beginner/saving_loading_models.html), click on the provided links.

---
# Create an Estimator

When a custom model is constructed in SageMaker, an entry point must be specified. This is the Python file which will be executed when the model is trained; the `train.py` function you specified above. To run a custom training script in SageMaker, construct an estimator, and fill in the appropriate constructor arguments:

* **entry_point**: The path to the Python script SageMaker runs for training and prediction.
* **source_dir**: The path to the training script directory `source_sklearn` OR `source_pytorch`.
* **role**: Role ARN, which was specified, above.
* **train_instance_count**: The number of training instances (should be left at 1).
* **train_instance_type**: The type of SageMaker instance for training. Note: Because Scikit-learn does not natively support GPU training, Sagemaker Scikit-learn does not currently support training on GPU instance types.
* **sagemaker_session**: The session used to train on Sagemaker.
* **hyperparameters** (optional): A dictionary `{'name':value, ..}` passed to the train function as hyperparameters.

Note: For a PyTorch model, there is another optional argument **framework_version**, which you can set to the latest version of PyTorch, `1.0`.

## EXERCISE: Define a Scikit-learn or PyTorch estimator

To import your desired estimator, use one of the following lines:
```
from sagemaker.sklearn.estimator import SKLearn
```
```
from sagemaker.pytorch import PyTorch
```

In [24]:
#pip install -U sagemaker

In [25]:
# NOT USING
"""
# your import and estimator code, here
from sagemaker.sklearn.estimator import SKLearn

estimator = SKLearn(entry_point="train.py",
                    source_dir="source_sklearn",
                    role=role,
                    train_instance_count=1,
                    framework_version='0.20.0',
                    py_version='py3',
                    train_instance_type='ml.c4.xlarge')
"""

'\n# your import and estimator code, here\nfrom sagemaker.sklearn.estimator import SKLearn\n\nestimator = SKLearn(entry_point="train.py",\n                    source_dir="source_sklearn",\n                    role=role,\n                    train_instance_count=1,\n                    framework_version=\'0.20.0\',\n                    py_version=\'py3\',\n                    train_instance_type=\'ml.c4.xlarge\')\n'

In [26]:
from sagemaker.pytorch import PyTorch
prefix = 'plagiarism_data'
output_path = 's3://{}/{}'.format(bucket, prefix)

# your import and estimator code, here
#pytorch_estimator = PyTorch(entry_point='train.py',
estimator = PyTorch(entry_point='train.py',
                    source_dir='source_pytorch',
                    train_instance_count=1, 
                   # train_instance_type='ml.c4.xlarge',
                    train_instance_type='ml.m4.xlarge',
                    output_path=output_path,
                    role=role,
                    framework_version='1.0',
                    py_version='py3',
                    sagemaker_session=sagemaker_session,
                    hyperparameters={
                        'input_features': 4,  # num of features
                        'hidden_dim': 30,
                        'output_dim': 1,
                        'epochs': 100 # could change to higher
                    })
                    


## EXERCISE: Train the estimator

Train your estimator on the training data stored in S3. This should create a training job that you can monitor in your SageMaker console.

In [28]:
%%time
input_data = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)

# Train your estimator on S3 training data
estimator.fit({'train': input_data})


'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


2021-05-19 01:54:24 Starting - Starting the training job...
2021-05-19 01:54:25 Starting - Launching requested ML instances......
2021-05-19 01:55:38 Starting - Preparing the instances for training......
2021-05-19 01:56:48 Downloading - Downloading input data...
2021-05-19 01:57:23 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2021-05-19 01:57:24,621 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
2021-05-19 01:57:24,624 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2021-05-19 01:57:24,640 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2021-05-19 01:57:27,676 sagemaker_pytorch_container.training INFO     Invoking user training script.
2021-05-19 01:57:28,075 sagemaker-containers INFO     Module train does not provide a setup.py. 
Generating setu

## EXERCISE: Deploy the trained model

After training, deploy your model to create a `predictor`. If you're using a PyTorch model, you'll need to create a trained `PyTorchModel` that accepts the trained `<model>.model_data` as an input parameter and points to the provided `source_pytorch/predict.py` file as an entry point. 

To deploy a trained model, you'll use `<model>.deploy`, which takes in two arguments:
* **initial_instance_count**: The number of deployed instances (1).
* **instance_type**: The type of SageMaker instance for deployment.

Note: If you run into an instance error, it may be because you chose the wrong training or deployment instance_type. It may help to refer to your previous exercise code to see which types of instances we used.

In [30]:
%%time

# importing PyTorchModel
from sagemaker.pytorch import PyTorchModel

# Create a model from the trained estimator data
# And point to the prediction script
model = PyTorchModel(model_data=estimator.model_data,
                     role = role,
                    framework_version='1.0',
                    py_version='py3',
                     entry_point='predict.py',
                     source_dir='source_pytorch')


# deploy and create a predictor

predictor = model.deploy(initial_instance_count=1, instance_type='ml.t2.medium')



Parameter image will be renamed to image_uri in SageMaker Python SDK v2.
'create_image_uri' will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


-------------------!CPU times: user 442 ms, sys: 12.2 ms, total: 455 ms
Wall time: 9min 32s


---
# Evaluating Your Model

Once your model is deployed, you can see how it performs when applied to our test data.

The provided cell below, reads in the test data, assuming it is stored locally in `data_dir` and named `test.csv`. The labels and features are extracted from the `.csv` file.

In [31]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
import os

# read in test data, assuming it is stored locally
test_data = pd.read_csv(os.path.join(data_dir, "test.csv"), header=None, names=None)

# labels are in the first column
test_y = test_data.iloc[:,0]
test_x = test_data.iloc[:,1:]

## EXERCISE: Determine the accuracy of your model

Use your deployed `predictor` to generate predicted, class labels for the test data. Compare those to the *true* labels, `test_y`, and calculate the accuracy as a value between 0 and 1.0 that indicates the fraction of test data that your model classified correctly. You may use [sklearn.metrics](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics) for this calculation.

**To pass this project, your model should get at least 90% test accuracy.**

In [32]:
# Note, test_x.values is a numpy nd array
# test_x is a dataframe
print(type(test_x),type(test_x.values),test_x)

<class 'pandas.core.frame.DataFrame'> <class 'numpy.ndarray'>            1         2         3         4
0   1.000000  0.984694  0.901042  0.820755
1   0.765306  0.709898  0.553633  0.621711
2   0.884444  0.526786  0.150000  0.597458
3   0.619048  0.250000  0.021739  0.427835
4   0.920000  0.675676  0.328571  0.775000
5   0.992674  0.985294  0.970149  0.993056
6   0.412698  0.112903  0.000000  0.346667
7   0.462687  0.070000  0.000000  0.189320
8   0.581152  0.142105  0.000000  0.247423
9   0.584211  0.142857  0.000000  0.294416
10  0.566372  0.151786  0.000000  0.258333
11  0.481481  0.171642  0.007692  0.278912
12  0.619792  0.251309  0.021390  0.341584
13  0.921739  0.842795  0.600000  0.929412
14  1.000000  0.980843  0.902724  1.000000
15  0.861538  0.396907  0.042105  0.504717
16  0.626168  0.465625  0.170886  0.558559
17  1.000000  0.992308  0.960938  0.996700
18  0.383838  0.076142  0.005181  0.178744
19  1.000000  0.989051  0.929630  0.854671
20  0.613924  0.133758  0.000000  0

In [33]:
# First: generate predicted, class labels
test_y_preds = None
test_y_preds = predictor.predict(test_x)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# test that your model generates the correct number of labels
assert len(test_y_preds)==len(test_y), 'Unexpected number of predictions.'
print('Test passed!')

Test passed!


In [34]:
# Second: calculate the test accuracy
from sklearn.metrics import accuracy_score
#accuracy = None
accuracy_score(test_y, test_y_preds)

print(accuracy_score)


## print out the array of predicted and true labels, if you want
print('\nPredicted class labels: ')
print(test_y_preds)
print('\nTrue class labels: ')
print(test_y.values)

<function accuracy_score at 0x7f46d628d510>

Predicted class labels: 
[1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0.
 0.]

True class labels: 
[1 1 1 1 1 1 0 0 0 0 0 0 1 1 1 1 1 1 0 1 0 1 1 0 0]


In [35]:
# calculate true positives, false positives, true negatives, false negatives
import numpy as np

tp = np.logical_and(test_y, test_y_preds).sum()
fp = np.logical_and(1-test_y, test_y_preds).sum()
tn = np.logical_and(1-test_y, 1-test_y_preds).sum()
fn = np.logical_and(test_y, 1-test_y_preds).sum()
print(tp, fp, tn, fn)

15 0 10 0


### Question 1: How many false positives and false negatives did your model produce, if any? And why do you think this is?

** Answer**: 
There are no false positive or false negative. I think we needed a larger dataset. 


### Question 2: How did you decide on the type of model to use? 

** Answer**: I selected 
binary classification. i wanted to work with a simple network. 
The input of features to produce 
a single sigmoid value, that can be rounded to a label: 0 or 1, as output. A simple network. Also working with Pytorch 
was a learning experience. 


----
## EXERCISE: Clean up Resources

After you're done evaluating your model, **delete your model endpoint**. You can do this with a call to `.delete_endpoint()`. You need to show, in this notebook, that the endpoint was deleted. Any other resources, you may delete from the AWS console, and you will find more instructions on cleaning up all your resources, below.

In [36]:
# uncomment and fill in the line below!
# <name_of_deployed_predictor>.delete_endpoint()
predictor.delete_endpoint()

### Deleting S3 bucket

When you are *completely* done with training and testing models, you can also delete your entire S3 bucket. If you do this before you are done training your model, you'll have to recreate your S3 bucket and upload your training data again.

In [37]:
# deleting bucket, uncomment lines below

bucket_to_delete = boto3.resource('s3').Bucket(bucket)
bucket_to_delete.objects.all().delete()

[{'ResponseMetadata': {'RequestId': '81PS14T37CP378FT',
   'HostId': 'X6S678eZ33CjcLql3AAkukk3bKd+jUeMtfTBmNzPKbv5udDIsxLCQ+7ojDhisf/iA9Oq6xGq83E=',
   'HTTPStatusCode': 200,
   'HTTPHeaders': {'x-amz-id-2': 'X6S678eZ33CjcLql3AAkukk3bKd+jUeMtfTBmNzPKbv5udDIsxLCQ+7ojDhisf/iA9Oq6xGq83E=',
    'x-amz-request-id': '81PS14T37CP378FT',
    'date': 'Wed, 19 May 2021 02:20:33 GMT',
    'content-type': 'application/xml',
    'transfer-encoding': 'chunked',
    'server': 'AmazonS3',
    'connection': 'close'},
   'RetryAttempts': 0},
  'Deleted': [{'Key': 'sagemaker-scikit-learn-2021-05-17-01-37-32-807/rule-output/ProfilerReport-1621215452/profiler-output/profiler-reports/LowGPUUtilization.json'},
   {'Key': 'plagiarism_data/pytorch-training-2021-05-12-22-31-50-346/rule-output/ProfilerReport-1620858710/profiler-output/profiler-reports/OverallSystemUsage.json'},
   {'Key': 'plagiarism_data/pytorch-training-2021-05-12-22-58-54-620/rule-output/ProfilerReport-1620860334/profiler-output/profiler-repo

### Deleting all your models and instances

When you are _completely_ done with this project and do **not** ever want to revisit this notebook, you can choose to delete all of your SageMaker notebook instances and models by following [these instructions](https://docs.aws.amazon.com/sagemaker/latest/dg/ex1-cleanup.html). Before you delete this notebook instance, I recommend at least downloading a copy and saving it, locally.

---
## Further Directions

There are many ways to improve or add on to this project to expand your learning or make this more of a unique project for you. A few ideas are listed below:
* Train a classifier to predict the *category* (1-3) of plagiarism and not just plagiarized (1) or not (0).
* Utilize a different and larger dataset to see if this model can be extended to other types of plagiarism.
* Use language or character-level analysis to find different (and more) similarity features.
* Write a complete pipeline function that accepts a source text and submitted text file, and classifies the submitted text as plagiarized or not.
* Use API Gateway and a lambda function to deploy your model to a web application.

These are all just options for extending your work. If you've completed all the exercises in this notebook, you've completed a real-world application, and can proceed to submit your project. Great job!